In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/frangr94/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/frangr94/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
df = pd.read_csv('data_api.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45451 entries, 0 to 45450
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   budget              45451 non-null  float64
 1   id                  45451 non-null  int64  
 2   original_language   45440 non-null  object 
 3   original_title      45451 non-null  object 
 4   overview            44510 non-null  object 
 5   popularity          45451 non-null  float64
 6   release_date        45451 non-null  object 
 7   revenue             45451 non-null  float64
 8   runtime             45205 non-null  float64
 9   status              45371 non-null  object 
 10  tagline             20425 non-null  object 
 11  title               45451 non-null  object 
 12  vote_average        45451 non-null  float64
 13  vote_count          45451 non-null  float64
 14  genres_unn          45451 non-null  object 
 15  langs_unn           45451 non-null  object 
 16  coll

In [26]:
# para este modelo no se utilizaran todas los campos: no me interesa generar predicciones en relacion, por ejemplo, a la popularidad, el origen o los
# presupuestos
df = df [['title','overview','release_year','original_language','genres_unn','prod_companies','cast_unn','directors','collection_unn']]

In [27]:
df.head()

,title,overview,release_year,original_language,genres_unn,prod_companies,cast_unn,directors,collection_unn
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995,en,"['Animation', 'Comedy', 'Family']",['Pixar Animation Studios'],"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",['John Lasseter'],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,1995,en,"['Adventure', 'Fantasy', 'Family']","['TriStar Pictures', 'Teitler Film', 'Intersco...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",['Joe Johnston'],no
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995,en,"['Romance', 'Comedy']","['Warner Bros.', 'Lancaster Gate']","['Walter Matthau', 'Jack Lemmon', 'Ann-Margret...",['Howard Deutch'],Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995,en,"['Comedy', 'Drama', 'Romance']",['Twentieth Century Fox Film Corporation'],"['Whitney Houston', 'Angela Bassett', 'Loretta...",['Forest Whitaker'],no
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995,en,['Comedy'],"['Sandollar Productions', 'Touchstone Pictures']","['Steve Martin', 'Diane Keaton', 'Martin Short...",['Charles Shyer'],Father of the Bride Collection


In [28]:
# solo se tomaran los primeros tres actores (considerados los principales) y el primer director
# funcion para desempaquetar las listas (eran strings)
def unpack_list(series):    
    lst=[]
    for i in series:
        z=i.replace('"', '')
        z=z.replace('[','')
        z=z.replace(']','')
        z=z.replace("'",'')
        z=z.split(',')
        lst.append(z[:3])
    
    return lst

df['genres']=unpack_list(df['genres_unn'])
df['prod']=unpack_list(df['prod_companies'])
df['cast']=unpack_list(df['cast_unn'])
df['director']=unpack_list(df['directors'])
df['director'] = df['director'].apply(lambda x: x[:1])
df['collection']=unpack_list(df['collection_unn'])

df.drop(columns=['genres_unn','prod_companies','cast_unn','directors','collection_unn'],inplace=True)

df.head()

,title,overview,release_year,original_language,genres,prod,cast,director,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995,en,"[Animation, Comedy, Family]",[Pixar Animation Studios],"[Tom Hanks, Tim Allen, Don Rickles]",[John Lasseter],[Toy Story Collection]
1,Jumanji,When siblings Judy and Peter discover an encha...,1995,en,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope ...","[Robin Williams, Jonathan Hyde, Kirsten Dunst]",[Joe Johnston],[no]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995,en,"[Romance, Comedy]","[Warner Bros., Lancaster Gate]","[Walter Matthau, Jack Lemmon, Ann-Margret]",[Howard Deutch],[Grumpy Old Men Collection]
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995,en,"[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],"[Whitney Houston, Angela Bassett, Loretta De...",[Forest Whitaker],[no]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995,en,[Comedy],"[Sandollar Productions, Touchstone Pictures]","[Steve Martin, Diane Keaton, Martin Short]",[Charles Shyer],[Father of the Bride Collection]


In [29]:
# se usa rake para extraer las palabras clave de overview
key_words = []
for i in df.overview:
 
    r = Rake()

    r.extract_keywords_from_text(str(i))

    key_words_dict_scores = r.get_word_degrees()

    key_words.append(list(key_words_dict_scores.keys()))

df['key_words']=key_words
df.drop(columns='overview',inplace=True)

In [30]:
def list_to_text(series):
    words_soup =[]
    for i in series:
        words=''
        for z in i:
            words = words+' '+z
        words_soup.append(words)  
    return words_soup 

In [31]:
def num_to_text(series):
    nums=[]
    for i in series:
        nums.append(str(i))
    return nums

In [32]:
# pasar todas las columnas a texto

df['genres_t'] = list_to_text(df['genres'])
df['prod_t'] = list_to_text(df['prod'])
df['cast_t'] = list_to_text(df['cast'])
df['director_t'] = list_to_text(df['director'])
df['collection_t'] = list_to_text(df['collection'])
df['key_words_t'] = list_to_text(df['key_words'])

# elimino columnas innecesarias
df.drop(columns=['genres','prod','cast','director','collection','key_words'],inplace=True)

In [33]:
# concatenar todas las columnas convertidas a texto en una sola
df['soup'] = df['original_language']+' '+' '+df['genres_t']+' '+df['prod_t']+' '+df['cast_t']+' '+df['director_t']+' '+df['collection_t']
#+' '+df['key_words_t']

In [34]:
# quitamos las columnas ya que no son necesarias a esta altura
df.drop(columns=['original_language','genres_t','prod_t','cast_t','director_t','collection_t','key_words_t'],inplace=True)

In [35]:
df.head()

,title,release_year,soup
0,Toy Story,1995,en Animation Comedy Family Pixar Animatio...
1,Jumanji,1995,en Adventure Fantasy Family TriStar Pictu...
2,Grumpier Old Men,1995,en Romance Comedy Warner Bros. Lancaster ...
3,Waiting to Exhale,1995,en Comedy Drama Romance Twentieth Century...
4,Father of the Bride Part II,1995,en Comedy Sandollar Productions Touchstone...


In [36]:
df.to_csv('data_modelado.csv',index=False)